***
# <font color=orange>**Cameron Peak Fire** </font>
***
## an analysis exploring high severity burned area on steep slopes <br> including potentially affected <font color=darkblue>watersheds</font>, with associated <font color=lightblue>streams</font> and <font color=blue>lakes</font>
#### *a notebook by Jesse Wooten, last edited on 12/14/20*
<br><br>

<img src="https://media.giphy.com/media/ZYJfwH8GLCs4VHVstA/giphy.gif" alt="Alt text that describes the graphic" title="Title text" />

In [ ]:
# written by Jesse Wooten
# for NR427 final projects
# December 14th, 2020

# This script creates a layer for high severity burned area from BAER data on the Cameron Peak Fire
# as well as a layer for steep slopes (> 30 %) derived from colorado DEM data
# finds the intersection of those two layers, where erosion risk is high
# the script then identifies which watersheds contain this high risk area
# as well as locating which streams flow through the risk zone, and which lakes are in affected watersheds
# two .csv reports and one .pdf plot of findings are created

In [ ]:
# installing a few libraries here
# i install all libraries the script uses, just in case you don't have some of them installed already 

!pip install descartes
!pip install adjustText
!pip install earthpy
!pip install pycrs
!pip install richdem
!pip install numpy
!pip install fiona
!pip install rasterio
!pip install shapely
!pip install os
!pip install requests
!pip install zipfile
!pip install json
!pip install ogr
!pip install gdal
!pip install matplotlib
!pip install pycrs

In [ ]:
# now we import

import richdem as rd
import adjustText as aT
import geopandas as gpd
import earthpy.spatial as es
import fiona, rasterio, shapely, os, requests, zipfile, json, matplotlib, descartes, pycrs, numpy, sys


In [ ]:
# some more specific importing

from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
from fiona.crs import from_epsg
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import zeros
from numpy import logical_and
from osgeo import gdal, ogr


## Let's access a few layers that are hosted online 
> #### go go gadget <font color=lightblue>streams</font> , <font color=blue>lakes</font> and <font color=darkblue>watersheds</font>!!!

In [ ]:
# Create data frames for some data we need

# Streams of Colorado

COstreamsURL = r'https://data.colorado.gov/api/geospatial/x238-vje7?method=export&format=GeoJSON'
COstreams = gpd.read_file(COstreamsURL)

print("CRS for Colorado Streams is " + str(COstreams.crs))

COstreams.plot()

In [ ]:
# Create lakes

COlakesURL = r'https://data.colorado.gov/api/geospatial/uksn-8qya?method=export&format=GeoJSON'
COlakes = gpd.read_file(COlakesURL)

COlakes.plot()
print("CRS for Colorado Lakes is " + str(COlakes.crs))



In [ ]:
# Create watersheds

COwsURL = r'https://geo.colorado.edu/download/file/47540-5ca269dcbfd55f000a1af25b-geojson.json'
COws = gpd.read_file(COwsURL)

COws.plot()
print("CRS for Colorado Watersheds is " + str(COws.crs))
COws.head()

## Now let's download a few zip files containing data we need
> #### go go gadget <font color=orange>*BAER burn severity* </font> data and <font color=pink>*Colorado DEM* </font>  data

In [ ]:
# Now... we are looking at all of the streams, lakes, and watersheds in the entire state of Colorado
# Seems like that scope may be a bit excessive for a wildfire - even a big one...
# Let's make a project boundary!!! (source: https://fsapps.nwcg.gov/baer/baer-imagery-support-data-download))

# to do this, we need to: 
# (1) make a new data folder 
# (2) download zips for burn severity data from the BAER website and NED data
# (3) unzip files







In [ ]:
# (1) This will create a new folder 

zipfiles = "zipfiles"
zippath = "zipcontents"
reports = "datareports"
newdata = "newlayers"

try:
    os.mkdir(zippath)
except OSError:
    print ("New folder creation for %s failed!!!" % zipfiles)
else:
    print ("Successfully created a folder for your .zip files , called %s " % zipfiles)
    
try:
    os.mkdir(zipfiles)
except OSError:
    print ("New folder creation for %s failed!!!" % zippath)
else:
    print ("Successfully created a folder for your zip data, called %s " % zippath)
       

try:
    os.mkdir(newdata)
except OSError:
    print ("New folder creation for %s failed!!!" % newdata)
else:
    print ("Successfully created a folder for additional layers you will create, called %s " % newdata)
    
try:
    os.mkdir(reports)
except OSError:
    print ("New folder creation for %s failed!!!" % reports)
else:
    print ("Successfully created a folder for reports we will print later, called %s " % reports)

In [ ]:
# (2) This downloads the zipped preliminary burn severity data from the USFS BAER site
# so that we can use the layers in our project!

zipurl = r'https://fsapps.nwcg.gov/baer/sites/baer/files/cameronpeak_co_preliminary_20200913.zip'
DEMurl = r'https://www.coloradoview.org/wp-content/coloradoviewData/aerial/dem-4-4.zip'

#files to be created
BAERzip = os.path.join(zipfiles, 'BAERdata.zip')
DEMzip = os.path.join(zipfiles, 'DEMdata.zip')

r = requests.get(zipurl, allow_redirects=True)
r2 = requests.get(DEMurl, allow_redirects=True)


open(BAERzip, 'wb').write(r.content)
open(DEMzip, 'wb').write(r2.content)

In [ ]:
# (3) This unzips the downloaded data into the "zipcontents" folder we created

with zipfile.ZipFile(BAERzip, 'r') as zip_ref:
    zip_ref.extractall(zippath)
    
with zipfile.ZipFile(DEMzip, 'r') as zip_ref:
    zip_ref.extractall(zippath)

In [ ]:
# Great!  Now we can bring in our freshly downloaded burn boundary layer
# We will deal with elevation a little later

burnpath = os.path.join(zippath, "co4060910587920200813_20200809_20200913_burn_bndy_utm.shp")
burnbound = gpd.read_file(burnpath)


burnbound.plot()

In [ ]:
# We need to identify affected watersheds, including their respective streams and lakes
# that may have been affected by the fire.

# First, we need to make sure all dataframes are in the same CRS
# We will use the CRS for the burn boundary as our reference CRS

print("The CRS for burn boundary is " + str(burnbound.crs))

In [ ]:

if COlakes.crs != burnbound.crs:
    print("Uhoh! CRS mismatch between lakes and buffered boundary... let's fix that.")
    COlakes = COlakes.to_crs(burnbound.crs)
    print("COlakes CRS is fixed and is now " + str(COlakes.crs) )
else:
    print("CRS for both lakes and buffered burn boundary is a match.")
    
if COstreams.crs != burnbound.crs:
    print("\nUhoh! CRS mismatch between rivers and buffered boundary... let's fix that.")
    COstreams = COstreams.to_crs(burnbound.crs)
    print("COstreams CRS is fixed and is now " + str(COstreams.crs) )
else:
    print("\nCRS for both streams and buffered burn boundary is a match.")
    
    
if COws.crs != burnbound.crs:
    print("\nUhoh! CRS mismatch between watersheds and buffered boundary... let's fix that.")
    COws = COws.to_crs(burnbound.crs)
    print("COws CRS is fixed and is now " + str(COws.crs) )
else:
    print("\nCRS for both watersheds and buffered burn boundary is a match.")
    
    

    
    
    


### ....now clipping streams, lakes and watersheds to our area of interest....

In [ ]:
# We can't just clip watersheds by the burn boundary, because we want to see the whole watershed
# that might be affected by the fire
# solution?  First make a layer of all watersheds that intersect with the fire
# unfortunately, we lose some of our pertinent data (i.e. watershed name) in this spatial join
# don't worry.... i have a plan

sheds_in_fire = gpd.sjoin(COws, burnbound, how="inner", op='intersects')
sheds_in_fire.plot()

# sheds_in_fire shows only the watersheds where some part falls within the fire

In [ ]:
# We will now clip the original CO watersheds layer by our 'sheds_in_fire'
# thereby creating a layer with the unique name of each affected watershed
# keep_geom_type = True prevents us from accidentally creating watersheds that are just boundary lines
# polygonal watersheds only, please!

sheds = gpd.clip(COws, sheds_in_fire, keep_geom_type=True)

sheds.plot(cmap='plasma')

In [ ]:
# Which watersheds may have been affected by the Cameron Peak Fire?

sheds_unique = sheds.Name.unique()

counter = 0 

for ws in sheds_unique:
    counter = counter + 1
    print(str(counter) + ". " + ws)

In [ ]:
# Any reservoir within the watershed may be at risk of sedimentation from erosion within the fire
# so we want to see any potentially affected streams and reservoirs
# to remedy this, we will use our new watersheds layer to clip streams and lakes


streams = gpd.clip(COstreams, sheds)
lakes = gpd.clip(COlakes, sheds)

streams.plot(figsize = (10,10))
lakes.plot(figsize = (10,10))



In [ ]:
# Let's save our clipped streams, lakes and watersheds layers to our new data folder, for future generations
# CPF = Cameron Peak Fire 

streams.to_file(os.path.join(newdata, "CPFstreams.shp"))
lakes.to_file(os.path.join(newdata, "CPFlakes.shp"))
sheds.to_file(os.path.join(newdata, "CPFwatersheds.shp"))

print("Shapefiles for clipped streams, lakes, and watersheds have been saved to your 'new layers' folder.")

## Now let's work with our dNBR and DEM rasters!

In [ ]:
# It's raster time!
# here are the paths to: 
# 1. the elevation tif (source: Geospatial Data Gateway)
# 2. DNBR burn severity tif (downloaded from BAER zip file earlier!!!)

# set variables for our rasters
elevrast = os.path.join(zippath, "4_4.tif")
burnDNBR = os.path.join(zippath, "co4060910587920200813_20200809_20200913_dnbr_utm.tif")


In [ ]:
# now let's open them up and take a peak at the CRS

elev = rasterio.open(elevrast)
burnsevras = rasterio.open(burnDNBR)

print("CRS for elevation is " + str(elev.crs))
print("CRS for burn severity is " + str(burnsevras.crs))

In [ ]:
# let's check if these are in the same CRS

if elev.crs == burnbound.crs and burnsevras.crs == burnbound.crs:
    print("Good news, your two rasters and project area have the same CRS!")
    print("Elev CRS is " + str(elev.crs) + " and burn sev CRS is " + str(burnsevras.crs))
else:
    print("Houston, we are mismatched.")
    elev = elev.to_crs(burnbound.crs)
    burnsevras = burnsevras.to_crs(burnbound.crs)
    print("Okay... now elev CRS is " + str(elev.crs) + " and burn sev CRS is " + str(burnsevras.crs))


In [ ]:
# well heck, what do these things look like?

show(elev, cmap='terrain')
show(burnsevras, cmap = 'jet')
    

In [ ]:
# We need to extract by mask to cut both of these rasters down to the size of our project area
# Note: burn sev raster needs to be extracted by mask to enable reclassification into burn sev categories


In [ ]:
# We have to convert our mask feature (burnbound) into json format, to please the fickle rasterio gods

def getFeatures(burnbound):
    # now turning this into rasterio friendly layer
    import json
    return [json.loads(burnbound.to_json())['features'][0]['geometry']]

coords = getFeatures(burnbound)

In [ ]:
# now... let's mask up

# Clip the raster with burn boundary polygon
camelev, out_transform = mask(dataset=elev, shapes=coords, crop = True)
camburnsevras, out_transform = mask(dataset=burnsevras, shapes=coords, crop=True)

In [ ]:
# Let's take a look at how these turned out....

show(camelev, cmap = 'terrain')
show(camburnsevras, cmap = 'jet')


In [ ]:
#update metadata for our new baby rasters


#elev_meta=  elev.meta.copy()
burn_meta=  burnsevras.meta.copy()
elev_meta = elev.meta.copy()

burnout = os.path.join(newdata, "burnsev_clip.tif")
elevout = os.path.join(newdata, "elev_clip.tif")

# now let's save this good stuff

with rasterio.open(elevout, "w", **elev_meta) as dest:
        dest.write(camelev)
        print("A copy of your clipped elevation raster has been saved at " + elevout)
with rasterio.open(burnout, "w", **burn_meta) as dest:
        dest.write(camburnsevras)
        print("A copy of your clipped burn severity raster has been saved at " + burnout)
        
                
       

## Raster reclassification time!
#### NOTE: instead of using the AGOL hosted Cam Peak burn severity vector layer<br>I decided instead to:
1. Use the raw dNBR .tif file from the BAER zip file we downloaded earlier
2. Reclassify the dNBR file into <font color=green>*unburned* </font>, <font color=gold>*low* </font>, <font color=orange>*moderate* </font>, and <font color=red>*high*</font> severity burned areas
3. Polygonize the reclassified raster
4. Select out the high severity area
<br><br><br>


In [ ]:
# Time to reclassify raster into our burn severity categories

# RECLASS code adapted from an unknown user, but 
# the source: http://geoexamples.blogspot.com/2013/06/gdal-performance-raster-classification.html


# To determine value cutoffs for different severity categories
# I started with the suggestions on page LA-38 
# from this document: https://www.fs.fed.us/rm/pubs/rmrs_gtr164/rmrs_gtr164_13_land_assess.pdf
# and then adjusted them until it looked similar to the BAER data

classification_values = [-692,100,500,800, 1130] #The interval values to classify  
classification_output_values = [1,2,3,4,0] #The value assigned to each interval  
  
    
in_file = burnout

#out file

burnreclass = os.path.join(newdata, "burnsev_reclassed.tif")

ds = gdal.Open(in_file)
band = ds.GetRasterBand(1)

block_sizes = band.GetBlockSize()
x_block_size = block_sizes[0]
y_block_size = block_sizes[1]

xsize = band.XSize
ysize = band.YSize

max_value = band.GetMaximum()
min_value = band.GetMinimum()

if max_value == None or min_value == None:
    stats = band.GetStatistics(0, 1)
    max_value = stats[1]
    min_value = stats[0]

format = "GTiff"
driver = gdal.GetDriverByName( format )
dst_ds = driver.Create(burnreclass, xsize, ysize, 1, gdal.GDT_Byte )
dst_ds.SetGeoTransform(ds.GetGeoTransform())
dst_ds.SetProjection(ds.GetProjection())

for i in range(0, ysize, y_block_size):
    if i + y_block_size < ysize:
        rows = y_block_size
    else:
        rows = ysize - i
    for j in range(0, xsize, x_block_size):
        if j + x_block_size < xsize:
            cols = x_block_size
        else:
            cols = xsize - j

        data = band.ReadAsArray(j, i, cols, rows)
        r = zeros((rows, cols), numpy.uint8)

        for k in range(len(classification_values) - 1):
            if classification_values[k] <= max_value and (classification_values[k + 1] > min_value ):
                r = r + classification_output_values[k] * logical_and(data >= classification_values[k], data < classification_values[k + 1])
        if classification_values[k + 1] < max_value:
            r = r + classification_output_values[k+1] * (data >= classification_values[k + 1])

        dst_ds.GetRasterBand(1).WriteArray(r,j,i)

dst_ds = None


In [ ]:
# This polygonizing code block adapted from Penn State
# https://www.e-education.psu.edu/geog489/node/2215

raster = gdal.Open(burnreclass)
raster.RasterCount
band = raster.GetRasterBand(1)

burnpoly = os.path.join(newdata, "burnsev_reclass_poly.shp")

drv = ogr.GetDriverByName('ESRI Shapefile')
outfile = drv.CreateDataSource(burnpoly) 
outlayer = outfile.CreateLayer('polygonized raster', srs = None )
newField = ogr.FieldDefn('DN', ogr.OFTReal)
outlayer.CreateField(newField)


# Once the shapefile is prepared, we call Polygonize(...) 
# and provide the band and the output layer as parameters plus a few additional parameters needed:


gdal.Polygonize(band, None, outlayer, 0, [])
outfile = None

In [ ]:
# loading polygonized raster into a geopandas dataframe

burnpolygeo = gpd.read_file(burnpoly)

In [ ]:

burnpolygeo.plot(column="DN", cmap= "YlOrRd", legend=True, figsize=(15,10))

In [ ]:
print(burnpolygeo.crs)

In [ ]:
# Welp.  We need matching CRS with other layers before we can proceed

burnpolygeo = burnpolygeo.set_crs(burnbound.crs)

print(burnpolygeo.crs)
burnpolygeo.to_file(os.path.join(newdata, "BurnSeverityCategories.shp"))
print("A new shapefile showing categorized burn severity has been saved to your 'newlayers' folder.")

In [ ]:
# Let's select out a new layer that is only High Severity (DN = 4.0)

highsev = burnpolygeo[burnpolygeo.DN == 4.0]
highsev.to_file(os.path.join(newdata, "HighSevBurnedArea.shp"))
print("A new shapefile showing high severity burned area has been saved to your 'newlayers' folder.")

In [ ]:
# Now let's see how it looks!

highsev.plot(facecolor = "DarkRed", figsize =(10,15))

## Let's hit the slopes!
### I mean, convert elevation to slope
#### but just for fun:

<img src="https://media.giphy.com/media/3o7WIHvqsj3emiv3eE/giphy.gif" alt="Alt text that describes the graphic" title="Title text" />



In [ ]:
# Okay... now about that elevation raster... what we really want is a slope raster
# to identify higher risk areas of erosion 
# which we are calling slopes >30% in high severity areas 

# so... let's turn elevation into slope.
# good thing we learned about richDEM, a package well suited for topographical raster magic

In [ ]:

# elevation to convert
dem = rd.LoadGDAL(elevout)


# resulting slope raster
outslope = os.path.join(newdata, "COslope_percent.tif")


slopey = rd.TerrainAttribute(dem, attrib='slope_percentage')
rd.rdShow(slopey, axes=False, cmap='YlOrBr', figsize=(8,5.5))


In [ ]:
#Save it!

rd.SaveGDAL(outslope, slopey)
print("A copy of the raw slope raster has been saved at " + outslope)

In [ ]:
# Now let's reclassify into slopes >= 30% (high erosion risk) and < 30% (lower risk)

# this RECLASS code adapted from an unknown user, but 
# the source: http://geoexamples.blogspot.com/2013/06/gdal-performance-raster-classification.html


classification_values = [-10,30] #The interval values to classify  
classification_output_values = [0,1] #The value assigned to each interval  
  
    
in_file = outslope

#out file

slopereclass = os.path.join(newdata, "sloperas_30.tif")

ds = gdal.Open(in_file)
band = ds.GetRasterBand(1)

block_sizes = band.GetBlockSize()
x_block_size = block_sizes[0]
y_block_size = block_sizes[1]

xsize = band.XSize
ysize = band.YSize

max_value = band.GetMaximum()
min_value = band.GetMinimum()

if max_value == None or min_value == None:
    stats = band.GetStatistics(0, 1)
    max_value = stats[1]
    min_value = stats[0]

format = "GTiff"
driver = gdal.GetDriverByName( format )
dst_ds = driver.Create(slopereclass, xsize, ysize, 1, gdal.GDT_Byte )
dst_ds.SetGeoTransform(ds.GetGeoTransform())
dst_ds.SetProjection(ds.GetProjection())

for i in range(0, ysize, y_block_size):
    if i + y_block_size < ysize:
        rows = y_block_size
    else:
        rows = ysize - i
    for j in range(0, xsize, x_block_size):
        if j + x_block_size < xsize:
            cols = x_block_size
        else:
            cols = xsize - j

        data = band.ReadAsArray(j, i, cols, rows)
        r = zeros((rows, cols), numpy.uint8)

        for k in range(len(classification_values) - 1):
            if classification_values[k] <= max_value and (classification_values[k + 1] > min_value ):
                r = r + classification_output_values[k] * logical_and(data >= classification_values[k], data < classification_values[k + 1])
        if classification_values[k + 1] < max_value:
            r = r + classification_output_values[k+1] * (data >= classification_values[k + 1])

        dst_ds.GetRasterBand(1).WriteArray(r,j,i)

dst_ds = None


In [ ]:
viewreclass = rasterio.open(slopereclass)
show(viewreclass)


In [ ]:

#this just shows that we have successfully reclassified into these 2 new values
# everything > 25% slope has the DN (digital number) of 1, everything else <25% is 0

show_hist(viewreclass, bins=50, lw=0.0, stacked=False, alpha=0.3,
      histtype='stepfilled', title="Histogram")

In [ ]:
# Now we need to polygonize our binary slope, so we can create a geopandas dataframe
# Of steep slopes to work with our other layers.


# This code was successfully used to polygonize burn severity,
# But failed horribly when trying to polygonize binary slope raster
# Despite various tweaks, reclassifications, and exhaustively searching for and trying several other code snippets

# This polygonizing code block adapted from Penn State
# https://www.e-education.psu.edu/geog489/node/2215


#band = raster.GetRasterBand(1)


#slopepoly = os.path.join(newdata, "slopere_poly.shp") 

#drv = ogr.GetDriverByName('ESRI Shapefile')
#outfile = drv.CreateDataSource(slopepoly) 
#outlayer = outfile.CreateLayer('polygonized raster', srs = None )
#newField = ogr.FieldDefn('DN', ogr.OFTReal)
#outlayer.CreateField(newField)


#gdal.Polygonize(band, None, outlayer, 0, [])
#outfile = None


In [ ]:
# What is our workaround, you ask?!!!
# Well, I converted the binary slope raster to vector format using ArcGIS Pro
# Converted that layer to a geojson file
# and am now hosting that geojson file (zipped) on Google Drive.
#
#
#
# Let's use it.

In [ ]:
# URL for the slope geojson I am hosting on google drive
slopezipurl = r'https://drive.google.com/u/0/uc?id=1-61JkGOCo--NjWH4gcBXXzqAe5GXGL1i&export=download'

slopezip = os.path.join(zipfiles, "slope_binary.zip")

r = requests.get(slopezipurl, allow_redirects=True)

open(slopezip, 'wb').write(r.content)


In [ ]:
# extracting zip file

with zipfile.ZipFile(slopezip, 'r') as zip_ref:
    zip_ref.extractall(zippath)
    print("Slope geojson has been saved in your zip folder.")

In [ ]:
# loading into geopandas

slopegeo = os.path.join(zippath, "slope_binary.geojson")
slopy = gpd.read_file(slopegeo)
slopy.head

In [ ]:
# Let's see it!

In [ ]:
slopy.plot(column="gridcode", cmap= "YlOrBr", legend=True, figsize=(15,10))

In [ ]:
# Let's select out a new layer that is only steep slopes (gridcode = 1)

steep = slopy[slopy.gridcode == 1]


In [ ]:
steep.plot(facecolor = "Orange", figsize =(10,15))

In [ ]:
# before we proceed with analyis, we have to do our favorite thing...
# verifying CRS

print("The current CRS for slope is " + str(steep.crs))

if steep.crs == burnbound.crs:
    print("Good news, slope has the same CRS as your other layers!!")
    print("And that CRS is " + str(steep.crs))
else:
    print("Houston, we are mismatched.  ")
    steep = steep.to_crs(burnbound.crs)
    print("Okay... now slope CRS has been converted to " + str(steep.crs))



# At this point in the script, we have created the following layers:
 - ## <font color=lightblue>*streams* </font>
     - accessed from online hosted layer
     - clipped to burn area
 - ## <font color=blue>*lakes* </font>
     - accessed from online hosted layer
     - clipped to burn area
 - ## <font color=darkblue>*watersheds* </font>
     - accessed from online hosted layer
     - selected to include all sub watersheds that overlap with the burn area
 - ## <font color=orange>*Cameron Peak Fire burn boundary* </font>
     - from downloaded/unzipped BAER data
 - ## <font color=red>*high severity burned area* </font>
     - reclassified from raw dNBR from downloaded/unzipped BAER data
     - categorized into severity classes
     - polygonized
     - new layer of 'high severity' created
 - ## <font color=brown>*steep slopes* </font>
     - rendered from downloaded/unzipped DEM raster
     - reclassified to show slopes >= 30%
     - polygonized (via google file)
     - new layer of 'steep slopes' created
     
<br>
<br>

### Now let's get to work.

<br>
<br>

In [ ]:
# Where do high severity burn areas intersect with steep slopes?

dangerzone = gpd.overlay(highsev, steep, how='intersection')

In [ ]:
dangerzone.plot(facecolor = "Red", figsize =(10,15))
plt.title("Danger Zone: high severity burned area occuring on steep slopes")

In [ ]:
#  Oooooh boy.
#  We should save this layer
dangerzone.to_file(os.path.join(newdata, "DangerZone.shp"))
print("A new shapefile showing the Danger Zone (high severity burn on steep slopes) is saved in the 'newlayers' folder.")

In [ ]:

#  Which watersheds contain the dangerzone?

dangersheds = gpd.sjoin(sheds, dangerzone, how="inner", op='intersects')

dangersheds.plot(cmap = 'plasma', figsize = (10,15))
plt.title("Watersheds containing the Danger Zone")

In [ ]:
# We will now clip our fire watersheds layer by our dangersheds
# thereby creating a layer with the unique name of each affected watershed
# keep_geom_type = True prevents us from accidentally creating watersheds that are just boundary lines
# polygonal watersheds only, please!

affectedsheds = gpd.clip(sheds, dangersheds, keep_geom_type=True)

affectedsheds.plot(cmap='plasma')

In [ ]:
# Which watersheds may have been affected by the Cameron Peak Fire?

danger_unique = affectedsheds.Name.unique()

counter = 0 

for danger in danger_unique:
    counter = counter + 1
    print(str(counter) + ". " + danger)

In [ ]:
# Wow.  
# Of the 18 watersheds within the fire, 16 contain some portion of overlapping high severity burn and steep slopes

# Let's save that watershed layer
affectedsheds.to_file(os.path.join(newdata, "WatershedsInDangerZone.shp"))
print("A new shapefile showing watersheds containing high severity burn on steep slopes is now located in 'newlayers' folder.")

In [ ]:
# Let's see where danger zone intersects with streams or lakes
dangerstreams = gpd.sjoin(streams, dangerzone, how="inner", op='intersects')
dangerlakes = gpd.sjoin(lakes, dangerzone, how="inner", op='intersects')

dangerstreams.plot(cmap = 'Blues', figsize = (10,15))
plt.title("Streams flowing through the danger zone")

# Note: this second plot will be blank. See comments in next cell. 
dangerlakes.plot(cmap = 'Blues', figsize = (10,10))
plt.title("Lakes in the danger zone... wait a second")

In [ ]:
# In retrospect, I suppose it makes sense that there are no lakes in areas with steep slopes... 
# Because the water would flow away.

# Let's save that layer of streams, though
dangerstreams.to_file(os.path.join(newdata, "StreamsInDangerZone.shp"))
print("A new shapefile of streams that flow through high risk zones is saved in the 'newlayers' folder")

In [ ]:
# Let's see which watersheds have streams that flowed directly through the danger zone

In [ ]:
# first we have to rename a troublesome field heading that prevents the join

dangerstreams = dangerstreams.rename(columns={'index_right': 'horse'})



In [ ]:
streamshed_risk = gpd.sjoin(affectedsheds, dangerstreams, how="left", op='contains')

affectedsheds.plot(cmap='plasma', figsize = (10,15))
plt.title("Affected watersheds")
streamshed_risk.plot(cmap = 'jet', figsize = (10,15))
plt.title("Affected watersheds with streams running directly through danger zone.")

In [ ]:
# Well looks like every watershed that contains high severity burn and steep slopes 
#   also, unfortunately, has at least one stream running through it that passes through the danger zone
#   What bodies of water lie within those watersheds?

atrisk_lakes = gpd.clip(lakes, affectedsheds)
atrisk_lakes.plot(cmap = 'Blues', figsize = (10,10))
plt.title("Bodies of water in affected watersheds")

In [ ]:
atrisk_lakes.head(10)

In [ ]:
# Let's see the names 

lakerisk_unique = atrisk_lakes.name.unique()

counter = 0 


for lago in lakerisk_unique:
    if str(lago) != "None":
        counter = counter + 1
        print(str(counter) + ". " + str(lago))
    else:
        pass

In [ ]:
# Let's convert the shape area of these lakes to Hectares
#And view the ten largest lakes



atrisk_lakes['hectares'] = atrisk_lakes['geometry'].area/ 10000

# Let's sort by the largest lakes
print("Largest lakes in affected watersheds:")
atrisk_lakes.sort_values(by = "hectares", ascending=False).drop(["co_fips", "shape_area", "shape_len",
                                                                "geometry"], axis = 1).head(15)
          

    

In [ ]:
# select out lakes with minimum surface of 10 hectares

biglakes = atrisk_lakes[atrisk_lakes.hectares > 10]

In [ ]:
print("Lakes at least 10 hectares in size")
biglakes.head(100)

In [ ]:
# Let's figure out the size of danger zone (high severity/steep) areas in each watershed

# first, we need to clean out some unused columns, so that the join gives us just the data we need
danger_simplify = dangerzone.drop([ 'FID', 'Id', 'gridcode'], axis=1)
shed_simplify = affectedsheds.drop(['id', 'OBJECTID', 'TNMID', 'MetaSource', "SourceData", "SourceOrig", "SourceFeat", "LoadDate", "GNIS_ID","AreaAcres", "AreaSqKm", "HUC12", "HUType", "HUMod", "ToHUC", "NonContrib", "NonContr_1", "Shape_Leng"], axis = 1)

# Identifying danger zones in watersheds
dangerinshed = gpd.sjoin(shed_simplify, danger_simplify, how= "right", op = "intersects")
dangerinshed.head()

In [ ]:
# Now, we want to group together the high sev / steep areas by watershed
shed_dissolve = dangerinshed.dissolve(by = "Name")

# and calculate the affected area
shed_dissolve['at_risk_area_hectares'] = shed_dissolve['geometry'].area/ 10000

shed_dissolve.head(20)

## We have now identified:
 - <font color=red>**"Danger Zone"** </font> i.e. areas that burned at high severity occurring on steep slopes
 - <font color=darkblue>**watersheds** </font> that contain some portion of the Danger Zone
 - <font color=lightblue>**streams** </font> that run through the Danger Zone
 - <font color=blue>**lakes** </font> in watersheds containing the Danger Zone
 
 <br>
 
 
### Our final tasks:
  - print .csv file showing total Danger Zone area within each water shed
  - print .csv file showing which lakes are in watersheds containing the Danger Zone
  - make a nice plot that shows all of this data together

In [ ]:
# Let's clean up the table so we can print a little report

shed_pre_csv = shed_dissolve.drop(['geometry', 'index_left', "States", 'Shape_Area', "DN"], axis = 1)

shed_pre_csv.head()


In [ ]:
shed_pre_csv.to_csv(os.path.join(reports,'AtRiskArea_byWatershed.csv'))
print("This just in: We created a .csv file containing a list of affected watersheds, along with the total at-risk area in each watershed that burned at high severity and is located on steep slopes.")
print("\nThis file is located in the reports folder we created earlier.")

In [ ]:
# Let's save a list of the water bodies larger than 10 hectares as a .csv file

risklakes_simplify = biglakes.drop(['co_fips', 'shape_area', 'shape_len'], axis=1)

lakesindanger = gpd.sjoin(shed_simplify, risklakes_simplify, how= "right", op = "intersects")
lakesindanger.head()

In [ ]:
lakes_precsv = lakesindanger.drop(['index_left', 'States', 'Shape_Area', 'geometry'], axis = 1)

In [ ]:
lakes_precsv.head()

In [ ]:
lakes_precsv.columns = ['Watershed', "Lake_Name", "Lake_Area_hectares"]
lakes_precsv.head()

In [ ]:
lakesindanger.to_file(os.path.join(newdata, "LakesinAffectedWatersheds.shp"))
lakes_precsv.to_csv(os.path.join(reports, r'AtRisk_Lakes.csv'))
print("This just in: We created a .csv file containing a list of the lakes > 10 hectares) in affected watersheds.")
print("This file is also located in the 'reports' folder we created earlier.")
print("\nWe also saved a shapefile of At Risk Lakes (in affected watersheds) in the 'newlayers' folder.")

In [ ]:
# Let's try to label the watersheds in our new figure

    
affectedsheds["center"] = affectedsheds["geometry"].centroid
shed_points = affectedsheds.copy()
shed_points.set_geometry("center", inplace = True)


In [ ]:
# This code labels each watersheds
# Adapted from...
# Source:  https://github.com/shotleft/how-to-python/blob/master/How%20it%20works%20-%20labelling%20districts%20in%20GeoPandas.ipynb



ax = affectedsheds.plot(figsize = (15, 12), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5)


# sets aspect to equal. This is done automatically
# when using *geopandas* plot on it's own, but not when
# working with pyplot directly.  
# Source:  https://geopandas.org/mapping.html


ax.set_aspect('equal')
ax.set_title('Lakes, At-risk Streams, and High Severity Burn area on Steep Slopes in Watersheds of Cameron Peak Fire')


texts = []

for x, y, label in zip(shed_points.geometry.x, shed_points.geometry.y, shed_points["Name"]):
    texts.append(plt.text(x, y, label, fontsize = 8))

aT.adjust_text(texts, force_points=0.3, force_text=0.8, expand_points=(1,1), expand_text=(1,1), 
               arrowprops=dict(arrowstyle="-", color='grey', lw=0.5))

# hide axis ticks/text
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)


# Now adding all lakes in affected sheds, but only the streams that run through the high-risk areas

dangerstreams.plot(ax=ax, marker = 'o',  color = 'cornflowerblue', markersize =10)


atrisk_lakes.plot(ax=ax, marker='o',  color='cornflowerblue', markersize=5)


dangerzone.plot(ax=ax, marker='o', color='red',  markersize=5)

# let's save this plot as well 
plt.savefig(os.path.join(reports, 'DangerZone.pdf'))  
print("We saved a copy of this map in your reports folder, for your viewing pleasure.")

## Take a look in the original folder where the .ipynb notebook file is located<br>You should see:
 - zip files for the DEM, BAER burn severity, and polygonized slope data downloaded from the internet
 - "zipcontents" folder containing all data that was unzipped from those files
 - "newlayers" folder containing .tif and .shp files created during the script that could be useful in the future
 - "reports" folder containing 
1. a lovely .pdf showing <font color=darkblue>**watersheds** </font>  in the <font color=red>**"Danger Zone"**</font>, <font color=lightblue>**streams** </font> that run through high risk areas, and <font color=blue>**lakes** </font> within the affected areas
2. a .csv report of total high severity / steep slope area within each watershed
3. a .csv report of lakes larger than 10 hectares within affected watersheds

     
<br>
<br>




<img src="https://media.giphy.com/media/l3vRaWnqG3gOZ8lsk/giphy.gif" />
     
     